# Input Data
### Input your file and select the year here:

In [ ]:
merged_data = "final_data.csv"
predict_yr = 2022

In [2]:
import pandas as pd
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
# read merged data
data = pd.read_csv(merged_data)
data['ID'] = data['ID'].astype(int)
data['Gift Date_y'] = pd.to_datetime(data['Gift Date_y'])

/var/folders/g5/t835zv0x7sb7qvk1j7k7t9y40000gp/T/ipykernel_65544/2179304840.py:1: DtypeWarning: Columns (13,14,15,17,23,70,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("final_data.csv")


In [3]:
# subset and filter data
data = data[data['Deceased']!='Yes'] # exclude dead ones
data = data[['ID',
       'Constituency Code','First Gift Amount',
        'cleaned_zip_codes', 'location',
       'total_gift', 'Total Gift Times',
       'newsletter_sub', 'tag_annual giving', 'tag_direct mail', 'tag_golf',
       'event_annual giving', 'event_book sale',
       'event_budget cut emergency appeal', 'event_calendar',
       'event_circle of love', 'event_direct mail', 'event_donor campaign',
       'event_face labels', 'event_general operating budget', 'event_golf',
       'event_holiday appeal opening doors', 'event_holiday cards',
       'event_holiday mailing', 'event_little city invitational',
       'event_newsletter', 'event_parent', 'event_pledge', 'event_raffle',
       'event_renewal', 'event_residential services', 'event_ride for smiles',
       'event_santa stickers', 'event_smiles campaign', 'event_teddy bear',
       'event_university of illinois work place giving', 'event_verne carson',
       'Direct Mail', 'Renewal','Gift Date_y','Gift Amount_y']]

## Feature Engineering

In [5]:
# Feature 1: type of event each
# for matching event, create a new column 'yesevent' for each row
data['yesevent'] = data.filter(like='event_').any(axis=1).astype(int)
# Create a new column 'total_event' that sums up 'yesevent' for each ID
data['total_event'] = data.groupby('ID')['yesevent'].transform('sum')

,ID,Constituency Code,First Gift Amount,cleaned_zip_codes,location,total_gift,Total Gift Times,newsletter_sub,tag_annual giving,tag_direct mail,...,event_smiles campaign,event_teddy bear,event_university of illinois work place giving,event_verne carson,Direct Mail,Renewal,Gift Date_y,Gift Amount_y,yesevent,total_event
0,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,0.0,...,0,0,0,0,1,0,1998-07-01,5.0,0,17
1,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,0,0,0,1,0,1999-09-30,20.0,1,17
2,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,0,0,0,1,0,2002-09-10,20.0,1,17
3,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,0,0,0,1,0,2003-12-29,25.0,1,17
4,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,0,0,0,1,0,2004-05-10,25.0,1,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519787,1274319,Individual,$560.00,60521,Not Chicago,660.0,1,NaN,0.0,0.0,...,0,0,0,0,0,0,2023-06-29,100.0,0,0
519788,1262892,Parent/Family/Guardian,$54.00,60193,Not Chicago,54.0,1,NaN,0.0,0.0,...,0,0,0,0,0,0,2023-06-30,54.0,1,1
519789,1274321,Individual,$150.00,60623,Chicago,150.0,1,NaN,0.0,0.0,...,0,0,0,0,0,0,2023-06-30,150.0,1,1
519790,1274320,Individual,$54.00,NaN,Not Chicago,54.0,1,NaN,0.0,0.0,...,0,0,0,0,0,0,2023-06-30,54.0,0,0


In [6]:
# Feature 2: exactly how many event
# Step 1: Filter out the relevant columns
event_columns = data.filter(like='event_')
# Calculate the attendance
attendance = event_columns.groupby(data['ID']).apply(lambda x: (x == 1).any())
total_events_attended = attendance.sum(axis=1)
# Convert the result to a DataFrame for merging
total_events_attended_df = total_events_attended.reset_index()
total_events_attended_df.columns = ['ID', 'types_of_event']
# Step 2: Merge this with the original DataFrame
data = data.merge(total_events_attended_df, on='ID', how='left')

,ID,Constituency Code,First Gift Amount,cleaned_zip_codes,location,total_gift,Total Gift Times,newsletter_sub,tag_annual giving,tag_direct mail,...,event_teddy bear,event_university of illinois work place giving,event_verne carson,Direct Mail,Renewal,Gift Date_y,Gift Amount_y,yesevent,total_event,types_of_event
0,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,0.0,...,0,0,0,1,0,1998-07-01,5.0,0,17,9
1,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,0,0,1,0,1999-09-30,20.0,1,17,9
2,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,0,0,1,0,2002-09-10,20.0,1,17,9
3,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,0,0,1,0,2003-12-29,25.0,1,17,9
4,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,0,0,1,0,2004-05-10,25.0,1,17,9


In [8]:
# Feature 3: Average event they attended per year
# Step 1: Create a new column for the year of the donation
data['Year'] = pd.to_datetime(data['Gift Date_y']).dt.year
# Step 2: Count the number of events attended for each donation
events_columns = data.columns[data.columns.str.startswith('event_')]
data['Num_Events'] = data[events_columns].sum(axis=1)
# Step 2a: Set Num_Events to 1 if the sum is equal or larger than 1, else set it to 0
data['Num_Events'] = data['Num_Events'].apply(lambda x: 1 if x >= 1 else 0)
# Step 3: Calculate the total number of events attended and the number of years donated for each donor
total_events_years = data.groupby('ID').agg({'Num_Events': 'sum', 'Year': 'nunique'}).reset_index()
# Step 4: Calculate the average number of events per year for each donor
total_events_years['Avg_Events_Per_Year'] = total_events_years['Num_Events'] / total_events_years['Year']
# Merge the average events per year back into the original DataFrame
data = pd.merge(data, total_events_years[['ID', 'Avg_Events_Per_Year']], on='ID', how='left')
# Drop the intermediate columns
data.drop(['Year', 'Num_Events'], axis=1, inplace=True)

,ID,Constituency Code,First Gift Amount,cleaned_zip_codes,location,total_gift,Total Gift Times,newsletter_sub,tag_annual giving,tag_direct mail,...,event_university of illinois work place giving,event_verne carson,Direct Mail,Renewal,Gift Date_y,Gift Amount_y,yesevent,total_event,types_of_event,Avg_Events_Per_Year
0,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,0.0,...,0,0,1,0,1998-07-01,5.0,0,17,9,1.545455
1,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,0,1,0,1999-09-30,20.0,1,17,9,1.545455
2,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,0,1,0,2002-09-10,20.0,1,17,9,1.545455
3,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,0,1,0,2003-12-29,25.0,1,17,9,1.545455
4,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,0,1,0,2004-05-10,25.0,1,17,9,1.545455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439400,1274319,Individual,$560.00,60521,Not Chicago,660.0,1,NaN,0.0,0.0,...,0,0,0,0,2023-06-29,100.0,0,0,0,0.000000
439401,1262892,Parent/Family/Guardian,$54.00,60193,Not Chicago,54.0,1,NaN,0.0,0.0,...,0,0,0,0,2023-06-30,54.0,1,1,1,1.000000
439402,1274321,Individual,$150.00,60623,Chicago,150.0,1,NaN,0.0,0.0,...,0,0,0,0,2023-06-30,150.0,1,1,1,1.000000
439403,1274320,Individual,$54.00,NaN,Not Chicago,54.0,1,NaN,0.0,0.0,...,0,0,0,0,2023-06-30,54.0,0,0,0,0.000000


In [9]:
# Feature 4: 
# Step 1: Get a new column for the number of different zip codes
num_unique_zips = data.groupby('ID')['cleaned_zip_codes'].nunique().reset_index()
num_unique_zips.rename(columns={'cleaned_zip_codes': 'Num_Different_Zips'}, inplace=True)
# Merge the number of unique zip codes back into the original DataFrame
data = pd.merge(data, num_unique_zips, on='ID', how='left')

,ID,Constituency Code,First Gift Amount,cleaned_zip_codes,location,total_gift,Total Gift Times,newsletter_sub,tag_annual giving,tag_direct mail,...,event_verne carson,Direct Mail,Renewal,Gift Date_y,Gift Amount_y,yesevent,total_event,types_of_event,Avg_Events_Per_Year,Num_Different_Zips
0,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,0.0,...,0,1,0,1998-07-01,5.0,0,17,9,1.545455,1
1,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,1,0,1999-09-30,20.0,1,17,9,1.545455,1
2,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,1,0,2002-09-10,20.0,1,17,9,1.545455,1
3,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,1,0,2003-12-29,25.0,1,17,9,1.545455,1
4,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,0,1,0,2004-05-10,25.0,1,17,9,1.545455,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439400,1274319,Individual,$560.00,60521,Not Chicago,660.0,1,NaN,0.0,0.0,...,0,0,0,2023-06-29,100.0,0,0,0,0.000000,1
439401,1262892,Parent/Family/Guardian,$54.00,60193,Not Chicago,54.0,1,NaN,0.0,0.0,...,0,0,0,2023-06-30,54.0,1,1,1,1.000000,1
439402,1274321,Individual,$150.00,60623,Chicago,150.0,1,NaN,0.0,0.0,...,0,0,0,2023-06-30,150.0,1,1,1,1.000000,1
439403,1274320,Individual,$54.00,NaN,Not Chicago,54.0,1,NaN,0.0,0.0,...,0,0,0,2023-06-30,54.0,0,0,0,0.000000,0


In [11]:
# Extremely high donor: top 50
total_contributions = data.groupby('ID')['total_gift'].sum()
sort = total_contributions.sort_values()
df_sorted = pd.DataFrame({'ID': sort.index, 'total_gift': sort.values})
top_50 = df_sorted.tail(50)
least_50 = df_sorted.head(50)
top_50

,ID,total_gift
39388,1263916,1.473093e+07
39389,1011660,1.515466e+07
39390,1262156,1.536160e+07
39391,1089129,1.621717e+07
39392,1245809,1.633523e+07
39393,1024443,1.690551e+07
39394,1013272,1.821147e+07
39395,1031509,1.844800e+07
39396,1088613,1.916625e+07
39397,1185839,1.948475e+07


**Threshold**

Low: <1000

Mid: 1000-9999

High: 100000-100000

Extremely High: >100000

In [12]:
# Extremely 
sum_by_id = data.groupby('ID')['total_gift'].sum()
# Filter sums that are greater than 1 million
sum_over_million = sum_by_id[sum_by_id > 1000000]
sum_over_million

ID
27784       3249435.00
27809      29420525.16
28568      93954683.08
1000107     1566300.00
1000109     1767985.20
              ...     
1265041     7097517.16
1266688     1895523.70
1269583    32579680.35
1270559     1044927.84
1273019     1633050.00
Name: total_gift, Length: 320, dtype: float64

In [13]:
# Feature 5: Biggest contribution made ever
biggest_id = total_contributions.idxmax()
biggest_amount = total_contributions.max()
print(f'The donor with ID {biggest_id} made the biggest contribution of {biggest_amount}')

The donor with ID 1098133 made the biggest contribution of 3717168379.4


In [14]:
# Feature 6: Smallest contribution made ever
smallest_id = total_contributions.idxmin()
smallest_amount = total_contributions.min()
print(f'The donor with ID {smallest_id} made the smallest contribution of {smallest_amount}')

The donor with ID 1271232 made the smallest contribution of 0.01


In [15]:
# Feature 7: Biggest contribution made in 5 years
max_gift_per_donor = data.groupby('ID')['total_gift'].max().reset_index()
# Rename the column to indicate it's the max gift amount
max_gift_per_donor.rename(columns={'total_gift': 'Max Gift Amount Since 2019'}, inplace=True)
# Merge this back into the original DataFrame
data = pd.merge(data, max_gift_per_donor, on='ID', how='left')

In [16]:
# Feature 8: Smallest contribution made in 5 years
min_gift_per_donor = data.groupby('ID')['total_gift'].min().reset_index()
# Rename the column to indicate it's the max gift amount
min_gift_per_donor.rename(columns={'total_gift': 'Min Gift Amount Since 2019'}, inplace=True)
# Merge this back into the original DataFrame
data = pd.merge(data, min_gift_per_donor, on='ID', how='left')

In [17]:
# Feature 9: Their favorite event(most attended event)
event_columns = ['event_annual giving', 'event_book sale', 'event_budget cut emergency appeal', 'event_calendar', 'event_circle of love', 'event_direct mail', 'event_donor campaign', 'event_face labels', 'event_general operating budget', 'event_golf', 'event_holiday appeal opening doors', 'event_holiday cards', 'event_holiday mailing', 'event_little city invitational', 'event_newsletter', 'event_parent', 'event_pledge', 'event_raffle', 'event_renewal', 'event_residential services', 'event_ride for smiles', 'event_santa stickers', 'event_smiles campaign', 'event_teddy bear', 'event_university of illinois work place giving', 'event_verne carson' ]
# Create a new DataFrame with only ID and event columns
df_events = data[['ID'] + event_columns]
# Group by 'ID' and sum each event column
event_counts_per_donor = df_events.groupby('ID').sum()
# subcategory
results_list = []
# Iterate over each row in event_counts_per_donor
for index, row in event_counts_per_donor.iterrows():
    # Find the maximum value and the corresponding event name
    max_event_count = row.max()
    max_event = row.idxmax()

    # Add to the results list
    results_list.append({'ID': index, 'MaxEvent': max_event, 'MaxEventCount': max_event_count})

# Convert the list of dictionaries to a DataFrame
results_sub = pd.DataFrame(results_list)
results_sub.rename(columns={'MaxEvent': 'Max Sub Event Attended'}, inplace=True)
data = pd.merge(data, results_sub[['ID','Max Sub Event Attended']], on='ID', how='left')

# major
annual_giving_events = ['event_annual giving', 'event_teddy bear', 'event_book sale', 'event_parent', 'event_general operating budget']
direct_mail_events = ['event_direct mail', 'event_circle of love', 'event_newsletter', 'event_pledge', 
                      'event_santa stickers', 'event_smiles campaign', 'event_holiday mailing', 
                      'event_verne carson', 'event_face labels', 'event_calendar', 'event_holiday cards', 'event_renewal', 'event_donor campaign']
golf_events = ['event_little city invitational']
event_counts_per_donor['annual_giving'] = event_counts_per_donor[annual_giving_events].sum(axis=1)
event_counts_per_donor['direct_mail'] = event_counts_per_donor[direct_mail_events].sum(axis=1)
event_counts_per_donor['golf'] = event_counts_per_donor[golf_events].sum(axis=1)
major_events_per_donor = event_counts_per_donor[['annual_giving','direct_mail','golf']]

results_list = []
# Iterate over each row in event_counts_per_donor
for index, row in major_events_per_donor.iterrows():
    # Find the maximum value and the corresponding event name
    max_event_count = row.max()
    max_event = row.idxmax()
    # Add to the results list
    results_list.append({'ID': index, 'MaxEvent': max_event, 'MaxEventCount': max_event_count})

# Convert the list of dictionaries to a DataFrame
results_major = pd.DataFrame(results_list)
results_major.rename(columns={'MaxEvent': 'Max Major Event Attended'}, inplace=True)
data = pd.merge(data, results_major[['ID','Max Major Event Attended']], on='ID', how='left')
# copy the data in case
data1 = data

In [20]:
# Feature 10: 10-year basis AVG gift amounts 
# Filter the DataFrame for years 2011 to 2021 and group by 'id', then calculate the mean
average_gift_10_years = data1[data1['Gift Date_y'].dt.year.between(predict_yr-10, predict_yr-1)].groupby('ID')['Gift Amount_y'].mean().reset_index()
# Merge the average_gift_10_years DataFrame with the original DataFrame
data1 = data1.merge(average_gift_10_years, on='ID', how='left', suffixes=('', '_avg_10_years'))
# Rename the column to 'average_gift_10_years'
data1.rename(columns={'gift_amount_y_avg_10_years': 'average_gift_10_years'}, inplace=True)
data1['Gift Amount_y_avg_10_years'].fillna(0,inplace=True)

/var/folders/g5/t835zv0x7sb7qvk1j7k7t9y40000gp/T/ipykernel_65544/3410141805.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data1['Gift Amount_y_avg_10_years'].fillna(0,inplace=True)


,ID,Constituency Code,First Gift Amount,cleaned_zip_codes,location,total_gift,Total Gift Times,newsletter_sub,tag_annual giving,tag_direct mail,...,total_event,types_of_event,Avg_Events_Per_Year,Num_Different_Zips,Max Gift Amount Since 2019,Min Gift Amount Since 2019,Max Sub Event Attended,Max Major Event Attended,total_2022,Gift Amount_y_avg_10_years
0,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,0.0,...,17,9,1.545455,1,410.0,410.0,event_general operating budget,direct_mail,0.0,0.0
1,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,17,9,1.545455,1,410.0,410.0,event_general operating budget,direct_mail,0.0,0.0
2,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,17,9,1.545455,1,410.0,410.0,event_general operating budget,direct_mail,0.0,0.0
3,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,17,9,1.545455,1,410.0,410.0,event_general operating budget,direct_mail,0.0,0.0
4,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,17,9,1.545455,1,410.0,410.0,event_general operating budget,direct_mail,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439400,1274319,Individual,$560.00,60521,Not Chicago,660.0,1,NaN,0.0,0.0,...,0,0,0.000000,1,660.0,660.0,event_annual giving,annual_giving,0.0,0.0
439401,1262892,Parent/Family/Guardian,$54.00,60193,Not Chicago,54.0,1,NaN,0.0,0.0,...,1,1,1.000000,1,54.0,54.0,event_newsletter,direct_mail,0.0,0.0
439402,1274321,Individual,$150.00,60623,Chicago,150.0,1,NaN,0.0,0.0,...,1,1,1.000000,1,150.0,150.0,event_annual giving,annual_giving,0.0,0.0
439403,1274320,Individual,$54.00,NaN,Not Chicago,54.0,1,NaN,0.0,0.0,...,0,0,0.000000,0,54.0,54.0,event_annual giving,annual_giving,0.0,0.0


In [21]:
# Feature 11: 10-year basis events attended
# Step 1: Filter the Data for the Last 10 Years
event_10_years = data1[data1['Gift Date_y'].dt.year.between(predict_yr-10, predict_yr-1)]
# Step 2: Identify Rows with Any Event
event_10_years['yesevent_10'] = event_10_years.filter(like='event_').any(axis=1).astype(int)
# Step 3: Create a new column 'total_event_10' that sums up 'yesevent_10' for each ID
event_10_years['total_event_10'] = event_10_years.groupby('ID')['yesevent_10'].transform('sum')
# Since we want to merge a unique 'total_event_10' value per 'ID', we need to drop duplicates
event_10_years_unique = event_10_years[['ID', 'total_event_10']].drop_duplicates()
# Step 4: Merge the 'total_event_10' back into the original DataFrame
data1 = data1.merge(event_10_years_unique, on='ID', how='left', suffixes=('', '_total_event_10'))
# Filled with 0 where NaN appears
data1['total_event_10'].fillna(0, inplace=True)

/var/folders/g5/t835zv0x7sb7qvk1j7k7t9y40000gp/T/ipykernel_65544/3056051728.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_10_years['yesevent_10'] = event_10_years.filter(like='event_').any(axis=1).astype(int)
/var/folders/g5/t835zv0x7sb7qvk1j7k7t9y40000gp/T/ipykernel_65544/3056051728.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_10_years['total_event_10'] = event_10_years.groupby('ID')['yesevent_10'].transform('sum')
/var/folders/g5/t835zv0x7sb7qvk1j7k7t9y40000gp/T/ipykernel_65544/3

,ID,Constituency Code,First Gift Amount,cleaned_zip_codes,location,total_gift,Total Gift Times,newsletter_sub,tag_annual giving,tag_direct mail,...,types_of_event,Avg_Events_Per_Year,Num_Different_Zips,Max Gift Amount Since 2019,Min Gift Amount Since 2019,Max Sub Event Attended,Max Major Event Attended,total_2022,Gift Amount_y_avg_10_years,total_event_10
0,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,0.0,...,9,1.545455,1,410.0,410.0,event_general operating budget,direct_mail,0.0,0.0,0.0
1,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,9,1.545455,1,410.0,410.0,event_general operating budget,direct_mail,0.0,0.0,0.0
2,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,9,1.545455,1,410.0,410.0,event_general operating budget,direct_mail,0.0,0.0,0.0
3,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,9,1.545455,1,410.0,410.0,event_general operating budget,direct_mail,0.0,0.0,0.0
4,1026027,Individual,$5.00,60453.0,Not Chicago,410.0,19,NaN,0.0,1.0,...,9,1.545455,1,410.0,410.0,event_general operating budget,direct_mail,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439400,1274319,Individual,$560.00,60521,Not Chicago,660.0,1,NaN,0.0,0.0,...,0,0.000000,1,660.0,660.0,event_annual giving,annual_giving,0.0,0.0,0.0
439401,1262892,Parent/Family/Guardian,$54.00,60193,Not Chicago,54.0,1,NaN,0.0,0.0,...,1,1.000000,1,54.0,54.0,event_newsletter,direct_mail,0.0,0.0,0.0
439402,1274321,Individual,$150.00,60623,Chicago,150.0,1,NaN,0.0,0.0,...,1,1.000000,1,150.0,150.0,event_annual giving,annual_giving,0.0,0.0,0.0
439403,1274320,Individual,$54.00,NaN,Not Chicago,54.0,1,NaN,0.0,0.0,...,0,0.000000,0,54.0,54.0,event_annual giving,annual_giving,0.0,0.0,0.0


In [ ]:
# Target feature: y
# Filter the DataFrame for input year and group by 'id' then sum the 'gift_amount_y'
target_year = data1[data1['Gift Date_y'].dt.year == predict_yr].groupby('ID')['Gift Amount_y'].sum().reset_index()
# Merge together
data1 = data1.merge(target_year, on='ID', how='left', suffixes=('', '_target_year'))
# Correctly rename the merged column to 'target_year'
data1.rename(columns={'Gift Amount_y_total_2022': 'target_year'}, inplace=True)
# Fill missing values in 'total_2022' with 0
data1['target_year'].fillna(0, inplace=True)

In [22]:
# Subset the final data
data2 = data1
data2 = data2[['ID', 'Constituency Code','location', 'total_gift', 'Total Gift Times', 'total_event',
       'types_of_event', 'Avg_Events_Per_Year', 'Num_Different_Zips',
       'Max Gift Amount Since 2019', 'Min Gift Amount Since 2019',
       'Max Sub Event Attended', 'Max Major Event Attended','total_2022','Gift Amount_y_avg_10_years','total_event_10']]
# Handle duplicate rows to make each row as a unique donor
# Step 1: Drop all duplicate rows
data2_unique = data2.drop_duplicates()
# Step 2: Find rows with the same 'ID': may occur more than once
duplicate_ids = data2_unique.groupby('ID').filter(lambda x: len(x) > 1)
# Keep the unique IDs (not the full rows)
duplicate_id_list = data2_unique.groupby('ID').filter(lambda x: len(x) > 1)['ID'].unique()
data2_unique.head()

,ID,Constituency Code,location,total_gift,Total Gift Times,total_event,types_of_event,Avg_Events_Per_Year,Num_Different_Zips,Max Gift Amount Since 2019,Min Gift Amount Since 2019,Max Sub Event Attended,Max Major Event Attended,total_2022,Gift Amount_y_avg_10_years,total_event_10
0,1026027,Individual,Not Chicago,410.0,19,17,9,1.545455,1,410.0,410.0,event_general operating budget,direct_mail,0.0,0.0,0.0
1,1026027,Individual,Not Chicago,410.0,19,17,9,1.545455,1,410.0,410.0,event_general operating budget,direct_mail,0.0,0.0,0.0
2,1026027,Individual,Not Chicago,410.0,19,17,9,1.545455,1,410.0,410.0,event_general operating budget,direct_mail,0.0,0.0,0.0
3,1026027,Individual,Not Chicago,410.0,19,17,9,1.545455,1,410.0,410.0,event_general operating budget,direct_mail,0.0,0.0,0.0
4,1026027,Individual,Not Chicago,410.0,19,17,9,1.545455,1,410.0,410.0,event_general operating budget,direct_mail,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439400,1274319,Individual,Not Chicago,660.0,1,0,0,0.000000,1,660.0,660.0,event_annual giving,annual_giving,0.0,0.0,0.0
439401,1262892,Parent/Family/Guardian,Not Chicago,54.0,1,1,1,1.000000,1,54.0,54.0,event_newsletter,direct_mail,0.0,0.0,0.0
439402,1274321,Individual,Chicago,150.0,1,1,1,1.000000,1,150.0,150.0,event_annual giving,annual_giving,0.0,0.0,0.0
439403,1274320,Individual,Not Chicago,54.0,1,0,0,0.000000,0,54.0,54.0,event_annual giving,annual_giving,0.0,0.0,0.0


In [25]:
# save the data
data2_unique.to_csv('data_new.csv', index=False)